# BigBasket Entire Product List
__Author__ : Mohammad Rouintan , 400222042

__Course__ : Undergraduate Data Science Course

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
basket = pd.read_csv('/kaggle/input/bigbasket-entire-product-list-28k-datapoints/BigBasket Products.csv',index_col='index')
basket.drop_duplicates(inplace=True)
basket.shape

(27201, 9)

In [3]:
basket.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
index,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [4]:
basket.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27201 entries, 1 to 27555
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product       27200 non-null  object 
 1   category      27201 non-null  object 
 2   sub_category  27201 non-null  object 
 3   brand         27200 non-null  object 
 4   sale_price    27201 non-null  float64
 5   market_price  27201 non-null  float64
 6   type          27201 non-null  object 
 7   rating        18738 non-null  float64
 8   description   27088 non-null  object 
dtypes: float64(3), object(6)
memory usage: 2.1+ MB


In [5]:
basket.describe().T

,count,mean,std,min,25%,50%,75%,max
sale_price,27201.0,320.943603,486.889562,2.45,95.0,190.0,351.0,12500.0
market_price,27201.0,380.356656,582.363754,3.00,100.0,220.0,425.0,12500.0
rating,18738.0,3.944434,0.735452,1.00,3.7,4.1,4.3,5.0


In [6]:
basket.columns

Index(['product', 'category', 'sub_category', 'brand', 'sale_price',
       'market_price', 'type', 'rating', 'description'],
      dtype='object')

In [7]:
basket.nunique()

product         23540
category           11
sub_category       90
brand            2313
sale_price       3256
market_price     1348
type              426
rating             40
description     21944
dtype: int64

In [8]:
basket.isnull().sum()

product            1
category           0
sub_category       0
brand              1
sale_price         0
market_price       0
type               0
rating          8463
description      113
dtype: int64

In [9]:
print('Percentage Null Data In Each Column')
print('*'*30)
total_count = basket.shape[0]
for col in basket.columns:
    null_count = basket[col].isnull().sum()
    print("{} : {:.2f}".format(col,null_count/total_count * 100))
    

Percentage Null Data In Each Column
******************************
product : 0.00
category : 0.00
sub_category : 0.00
brand : 0.00
sale_price : 0.00
market_price : 0.00
type : 0.00
rating : 31.11
description : 0.42


In [10]:
print('Total Null Data')
null_count = basket.isnull().sum().sum()
total_count = np.product(basket.shape)
print("{:.2f}".format(null_count/total_count * 100))

Total Null Data
3.50


In [11]:
basket = basket.dropna()
basket.shape

(18650, 9)

In [12]:
basket.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
index,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


### Category

In [13]:
basket['category'].describe()

count                18650
unique                   9
top       Beauty & Hygiene
freq                  5352
Name: category, dtype: object

In [14]:
category = pd.DataFrame(basket.groupby(['category']).size()).reset_index()
category = category.rename(columns={0: 'NumberOfProducts'})
category

,category,NumberOfProducts
0,Baby Care,495
1,"Bakery, Cakes & Dairy",665
2,Beauty & Hygiene,5352
3,Beverages,630
4,Cleaning & Household,2075
5,"Foodgrains, Oil & Masala",2170
6,Gourmet & World Food,2357
7,"Kitchen, Garden & Pets",2442
8,Snacks & Branded Foods,2464


In [15]:
sorted_category = category.sort_values(by=['NumberOfProducts'], ascending=False)
sorted_category

,category,NumberOfProducts
2,Beauty & Hygiene,5352
8,Snacks & Branded Foods,2464
7,"Kitchen, Garden & Pets",2442
6,Gourmet & World Food,2357
5,"Foodgrains, Oil & Masala",2170
4,Cleaning & Household,2075
1,"Bakery, Cakes & Dairy",665
3,Beverages,630
0,Baby Care,495


In [16]:
px.bar(data_frame=sorted_category,
 x='category',
 y='NumberOfProducts',
 color='NumberOfProducts',
 color_continuous_scale='reds',
 title=f'Count of Products in Each Category')

### Sub_category

In [17]:
basket['sub_category'].describe()

count         18650
unique           77
top       Skin Care
freq           1601
Name: sub_category, dtype: object

In [18]:
sub_category = pd.DataFrame(basket.groupby(['sub_category']).size()).reset_index()
sub_category = sub_category.rename(columns={0: 'NumberOfProducts'})
sub_category

,sub_category,NumberOfProducts
0,All Purpose Cleaners,305
1,Appliances & Electricals,124
2,"Atta, Flours & Sooji",118
3,Baby Accessories,27
4,Baby Bath & Hygiene,166
...,...,...
72,Steel Utensils,265
73,Storage & Accessories,652
74,Tea,269
75,Tinned & Processed Food,127


In [19]:
sorted_sub_category = sub_category.sort_values(by=['NumberOfProducts'], ascending=False)
sorted_sub_category

,sub_category,NumberOfProducts
67,Skin Care,1601
8,Bath & Hand Wash,795
42,Hair Care,777
49,Masalas & Spices,763
73,Storage & Accessories,652
...,...,...
6,Bakery Snacks,24
52,Mothers & Maternity,23
76,Water,16
41,Gourmet Breads,10


In [20]:
px.bar(data_frame=sorted_sub_category[:20],
 x='sub_category',
 y='NumberOfProducts',
 color='NumberOfProducts',
 color_continuous_scale='greens',
 title=f'Count of Products in Top 20 Sub_Category')

### Brand

In [21]:
basket['brand'].describe()

count        18650
unique        1933
top       bb Royal
freq           278
Name: brand, dtype: object

In [22]:
brand = pd.DataFrame(basket.groupby(['brand']).size()).reset_index()
brand = brand.rename(columns={0: 'NumberOfProducts'})
brand

,brand,NumberOfProducts
0,&Me,3
1,&Stirred,5
2,109°F,1
3,137 Degree,1
4,18 Herbs,22
...,...,...
1928,kwality walls,15
1929,mJOY,1
1930,marshmallow,2
1931,mother dairy,13


In [23]:
sorted_brand = brand.sort_values(by=['NumberOfProducts'], ascending=False)
sorted_brand

,brand,NumberOfProducts
1919,bb Royal,278
138,BB Home,172
78,Amul,153
751,Himalaya,139
146,BIOTIQUE,102
...,...,...
431,Delfi Twister,1
430,Delfi,1
1531,Sanjeev Kapoor,1
216,Blor,1


In [24]:
px.bar(data_frame=sorted_brand[:20],
 x='brand',
 y='NumberOfProducts',
 color='NumberOfProducts',
 color_continuous_scale='blues',
 title=f'Count of Products in Top 20 Brand')

### SalePrice & MarketPrice

In [25]:
basket['sale_price'].describe()

count    18650.000000
mean       266.552195
std        363.974002
min          3.000000
25%         85.000000
50%        165.000000
75%        300.000000
max       6660.000000
Name: sale_price, dtype: float64

In [26]:
basket['market_price'].describe()

count    18650.000000
mean       320.816505
std        458.994859
min          3.000000
25%         95.000000
50%        195.000000
75%        360.000000
max       7400.000000
Name: market_price, dtype: float64

In [27]:
sale_price = go.Box(x=basket['sale_price'], name='sale_price')
market_price = go.Box(x=basket['market_price'], name='market_price')

fig = go.Figure()
fig.add_trace(sale_price)
fig.add_trace(market_price)
fig.show()

In [28]:
sale_price = go.Violin(x=basket['sale_price'], name='sale_price')
market_price = go.Violin(x=basket['market_price'], name='market_price')

fig = go.Figure()
fig.add_trace(sale_price)
fig.add_trace(market_price)
fig.show()

### Type

In [29]:
basket['type'].describe()

count         18650
unique          358
top       Face Care
freq           1070
Name: type, dtype: object

In [30]:
product_type = pd.DataFrame(basket.groupby(['type']).size()).reset_index()
product_type = product_type.rename(columns={0: 'NumberOfProducts'})
product_type

,type,NumberOfProducts
0,Adult Diapers,20
1,"Agarbatti, Incense Sticks",85
2,Air Freshener,118
3,Almonds,5
4,"Aluminium Foil, Clingwrap",18
...,...,...
353,"Wet Wipe, Pocket Tissues",4
354,Whole Spices,87
355,Wine & Rice Vinegar,4
356,Women's Deodorants,109


In [31]:
sorted_product_type = product_type.sort_values(by=['NumberOfProducts'], ascending=False)
sorted_product_type

,type,NumberOfProducts
131,Face Care,1070
223,Men's Deodorants,402
303,Shampoo & Conditioner,379
41,Blended Masalas,343
83,Containers Sets,330
...,...,...
21,Baby Toys,1
23,Bagels & Baguette,1
161,Gourmet Tea & Tea Bags,1
176,Health Supplements,1


In [32]:
px.bar(data_frame=sorted_product_type[:20],
 x='type',
 y='NumberOfProducts',
 color='NumberOfProducts',
 color_continuous_scale='purples',
 title=f'Count of Products in Top 20 Type')

### Rating

In [33]:
basket['rating'].describe()

count    18650.000000
mean         3.944080
std          0.736008
min          1.000000
25%          3.700000
50%          4.100000
75%          4.300000
max          5.000000
Name: rating, dtype: float64

In [34]:
box = go.Box(x=basket['rating'], name='Box_of_rating')
violin = go.Violin(x=basket['rating'], name='Violin_of_rating')

fig = go.Figure()
fig.add_trace(box)
fig.add_trace(violin)
fig.show()

In [35]:
basket.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
index,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [36]:
basket.loc[3, 'description']

'A perfect gift for all occasions, be it your mother, sister, in-laws, boss or your friends, this beautiful designer piece wherever placed, is sure to beautify the surroundings Traditional design This type diya has been used for Diwali and All other Festivals for centuries. Sturdy and easy to carry The feet keep it balanced to ensure safety. Wonderful Oil Lamp made in Brass also called as Jyoti. This is a handcrafted piece of Indian brass Deepak.'

In [37]:
basket['combine_some_features'] = basket['description'] + ' ' + basket['category'] + ' ' + basket['sub_category'] + ' ' + basket['brand'] + ' ' + basket['type']

In [38]:
basket.loc[3, 'combine_some_features']

'A perfect gift for all occasions, be it your mother, sister, in-laws, boss or your friends, this beautiful designer piece wherever placed, is sure to beautify the surroundings Traditional design This type diya has been used for Diwali and All other Festivals for centuries. Sturdy and easy to carry The feet keep it balanced to ensure safety. Wonderful Oil Lamp made in Brass also called as Jyoti. This is a handcrafted piece of Indian brass Deepak. Cleaning & Household Pooja Needs Trm Lamp & Lamp Oil'

In [39]:
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), min_df=0)
tfidf_matrix = tfidf.fit_transform(basket['combine_some_features'])
tfidf_matrix.shape

(18650, 330431)

In [40]:
cosine_sim_tfidf_vec = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim_tfidf_vec

array([[1.        , 0.00392592, 0.00667172, ..., 0.00331355, 0.00271151,
        0.00460339],
       [0.00392592, 1.        , 0.0021501 , ..., 0.0015049 , 0.        ,
        0.        ],
       [0.00667172, 0.0021501 , 1.        , ..., 0.0024738 , 0.        ,
        0.        ],
       ...,
       [0.00331355, 0.0015049 , 0.0024738 , ..., 1.        , 0.00281518,
        0.        ],
       [0.00271151, 0.        , 0.        , ..., 0.00281518, 1.        ,
        0.        ],
       [0.00460339, 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [43]:
product_names = pd.Series(basket.index, index=basket['product']).drop_duplicates()
product_names

product
Garlic Oil - Vegetarian Capsule 500 mg                         1
Water Bottle - Orange                                          2
Brass Angle Deep - Plain, No.2                                 3
Cereal Flip Lid Container/Storage Jar - Assorted Colour        4
Creme Soft Soap - For Hands & Body                             5
                                                           ...  
Wottagirl! Perfume Spray - Heaven, Classic                 27551
Rosemary                                                   27552
Peri-Peri Sweet Potato Chips                               27553
Green Tea - Pure Original                                  27554
United Dreams Go Far Deodorant                             27555
Name: index, Length: 18650, dtype: int64

In [42]:
def recommender_with_description_and_some_features(title, recommend_number=10):
    idx = product_names[title]
    sim_scores = list(enumerate(cosine_sim_tfidf_vec[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:recommend_number + 1]
    prodcut_indices = [i[0] for i in sim_scores]
    return basket['product'].iloc[prodcut_indices]

In [44]:
recommender_with_description_and_some_features('Garlic Oil - Vegetarian Capsule 500 mg')

index
11321    Rectangular Plastic Container - With Lid, Mult...
11643                               Jar - With Lid, Yellow
26452     Round & Flat Storage Container - With lid, Green
6164     Premium Rectangular Plastic Container With Lid...
9547     Premium Round Plastic Container With Lid - Yellow
13960    Premium Rectangular Plastic Container With Lid...
19382    Premium Round & Flat Storage Container With Li...
24256      Premium Round Plastic Container With Lid - Blue
26068    Premium Round Plastic Container With Lid - Mul...
26075      Premium Round Plastic Container With Lid - Pink
Name: product, dtype: object

In [45]:
recommender_with_description_and_some_features('Cadbury Perk - Chocolate Bar')

index
7044     X-press Instant Noodles - Masala Delight, Supe...
17373                      1-2-3 Noodles - Pure Vegetarian
26766                          Noodles - Chinese Hakka Veg
25404                      1-2-3 Noodles - Pure Vegetarian
19531                   1-2-3 Noodles - Veg Masala Flavour
20242                      1-2-3 Noodles - Chicken Flavour
9486                    1-2-3 Noodles - Veg Masala Flavour
21401                      1-2-3 Noodles - Chicken Flavour
9092                                Noodles - Fiery Chilli
15618                         FunFoods Pasta & Pizza Sauce
Name: product, dtype: object

In [46]:
basket.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description,combine_some_features
index,,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ...","Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m...","A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...,Nivea Creme Soft Soap gives your skin the best...


In [47]:
copy_basket = basket.copy()

In [48]:
strip = lambda a:a.strip()
convert_to_list = lambda a:list(map(strip, re.split('& |, |\*|\n', a)))

In [49]:
for col in ['category', 'sub_category', 'type']:
    copy_basket[col] = copy_basket[col].apply(convert_to_list)

In [50]:
copy_basket.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description,combine_some_features
index,,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,"[Beauty, Hygiene]",[Hair Care],Sri Sri Ayurveda,220.0,220.0,"[Hair Oil, Serum]",4.1,This Product contains Garlic Oil that is known...,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"[Kitchen, Garden, Pets]","[Storage, Accessories]",Mastercook,180.0,180.0,"[Water, Fridge Bottles]",2.3,"Each product is microwave safe (without lid), ...","Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2","[Cleaning, Household]",[Pooja Needs],Trm,119.0,250.0,"[Lamp, Lamp Oil]",3.4,"A perfect gift for all occasions, be it your m...","A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,"[Cleaning, Household]","[Bins, Bathroom Ware]",Nakoda,149.0,176.0,"[Laundry, Storage Baskets]",3.7,Multipurpose container with an attractive desi...,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,"[Beauty, Hygiene]","[Bath, Hand Wash]",Nivea,162.0,162.0,"[Bathing Bars, Soaps]",4.4,Nivea Creme Soft Soap gives your skin the best...,Nivea Creme Soft Soap gives your skin the best...


In [51]:
def cleaner(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [52]:
for col in ['category', 'sub_category', 'type','brand']:
    copy_basket[col] = copy_basket[col].apply(cleaner)

In [53]:
copy_basket.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description,combine_some_features
index,,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,"[beauty, hygiene]",[haircare],srisriayurveda,220.0,220.0,"[hairoil, serum]",4.1,This Product contains Garlic Oil that is known...,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"[kitchen, garden, pets]","[storage, accessories]",mastercook,180.0,180.0,"[water, fridgebottles]",2.3,"Each product is microwave safe (without lid), ...","Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2","[cleaning, household]",[poojaneeds],trm,119.0,250.0,"[lamp, lampoil]",3.4,"A perfect gift for all occasions, be it your m...","A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,"[cleaning, household]","[bins, bathroomware]",nakoda,149.0,176.0,"[laundry, storagebaskets]",3.7,Multipurpose container with an attractive desi...,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,"[beauty, hygiene]","[bath, handwash]",nivea,162.0,162.0,"[bathingbars, soaps]",4.4,Nivea Creme Soft Soap gives your skin the best...,Nivea Creme Soft Soap gives your skin the best...


In [54]:
def combine_categorical_features(x):
    return ' '.join(x['category']) + ' ' + ' '.join(x['sub_category']) + ' ' + x['brand'] + ' ' + ' '.join(x['type'])
copy_basket['combine_categorical_features'] = copy_basket.apply(combine_categorical_features, axis=1)

In [55]:
copy_basket['combine_categorical_features'].head()

index
1    beauty hygiene haircare srisriayurveda hairoil...
2    kitchen garden pets storage accessories master...
3       cleaning household poojaneeds trm lamp lampoil
4    cleaning household bins bathroomware nakoda la...
5    beauty hygiene bath handwash nivea bathingbars...
Name: combine_categorical_features, dtype: object

In [56]:
copy_basket.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description,combine_some_features,combine_categorical_features
index,,,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,"[beauty, hygiene]",[haircare],srisriayurveda,220.0,220.0,"[hairoil, serum]",4.1,This Product contains Garlic Oil that is known...,This Product contains Garlic Oil that is known...,beauty hygiene haircare srisriayurveda hairoil...
2,Water Bottle - Orange,"[kitchen, garden, pets]","[storage, accessories]",mastercook,180.0,180.0,"[water, fridgebottles]",2.3,"Each product is microwave safe (without lid), ...","Each product is microwave safe (without lid), ...",kitchen garden pets storage accessories master...
3,"Brass Angle Deep - Plain, No.2","[cleaning, household]",[poojaneeds],trm,119.0,250.0,"[lamp, lampoil]",3.4,"A perfect gift for all occasions, be it your m...","A perfect gift for all occasions, be it your m...",cleaning household poojaneeds trm lamp lampoil
4,Cereal Flip Lid Container/Storage Jar - Assort...,"[cleaning, household]","[bins, bathroomware]",nakoda,149.0,176.0,"[laundry, storagebaskets]",3.7,Multipurpose container with an attractive desi...,Multipurpose container with an attractive desi...,cleaning household bins bathroomware nakoda la...
5,Creme Soft Soap - For Hands & Body,"[beauty, hygiene]","[bath, handwash]",nivea,162.0,162.0,"[bathingbars, soaps]",4.4,Nivea Creme Soft Soap gives your skin the best...,Nivea Creme Soft Soap gives your skin the best...,beauty hygiene bath handwash nivea bathingbars...


In [57]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(copy_basket['combine_categorical_features'])

In [58]:
cosine_sim_count_vec = cosine_similarity(count_matrix, count_matrix)
cosine_sim_count_vec

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.27216553],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.27216553, 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [59]:
copy_basket = copy_basket.reset_index()
indices = pd.Series(copy_basket.index, index=copy_basket['product'])

In [60]:
def recommender_with_categorical_features(title, recommend_number=10):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim_count_vec[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:recommend_number + 1]
    product_indices = [i[0] for i in sim_scores]
    return copy_basket['product'].iloc[product_indices]

In [62]:
recommender_with_categorical_features('Garlic Oil - Vegetarian Capsule 500 mg')

521      Evening Primrose Oil - Vegetarian Capsule (500...
7172                Brahmi Bhringaraj Taila - Anti Graying
13395                     Sukesha Taila - for Healthy Hair
133                                      Pain Relief - Oil
252                                   Hair Spa Oil Therapy
317                             Hair Serum - Anti-Dandruff
332                                  Hair Fall Control Oil
790                                Hair Oil - Pigmentation
904                            Natural Nourishing Hair Oil
1253                          Onion Herbal Hair Growth Oil
Name: product, dtype: object

In [61]:
recommender_with_categorical_features('Cadbury Perk - Chocolate Bar')

961                               Nutties Chocolate Pack
978                                 5 Star Chocolate Bar
1089            Dairy Milk Silk - Hazelnut Chocolate Bar
2069    Perk - Chocolate, Home Treats, 175.5 g, 27 Units
5184                             Dark Milk Chocolate Bar
5470              Dairy Milk Silk Mousse - Chocolate Bar
5965                             Dark Milk Chocolate Bar
6509                                Chocolate Bar - Fuse
7680                               Choclairs Gold Coffee
8822         5 Star Chocolate Home Pack, 200 g, 20 units
Name: product, dtype: object

In [63]:
recommender1 = recommender_with_description_and_some_features('Garlic Oil - Vegetarian Capsule 500 mg').values
recommender2 = recommender_with_categorical_features('Garlic Oil - Vegetarian Capsule 500 mg').values

pd.DataFrame({'Recommender1': recommender1,'Recommender2': recommender2})

,Recommender1,Recommender2
0,"Rectangular Plastic Container - With Lid, Mult...",Evening Primrose Oil - Vegetarian Capsule (500...
1,"Jar - With Lid, Yellow",Brahmi Bhringaraj Taila - Anti Graying
2,"Round & Flat Storage Container - With lid, Green",Sukesha Taila - for Healthy Hair
3,Premium Rectangular Plastic Container With Lid...,Pain Relief - Oil
4,Premium Round Plastic Container With Lid - Yellow,Hair Spa Oil Therapy
5,Premium Rectangular Plastic Container With Lid...,Hair Serum - Anti-Dandruff
6,Premium Round & Flat Storage Container With Li...,Hair Fall Control Oil
7,Premium Round Plastic Container With Lid - Blue,Hair Oil - Pigmentation
8,Premium Round Plastic Container With Lid - Mul...,Natural Nourishing Hair Oil
9,Premium Round Plastic Container With Lid - Pink,Onion Herbal Hair Growth Oil


In [64]:
recommender1 = recommender_with_description_and_some_features('Cadbury Perk - Chocolate Bar').values
recommender2 = recommender_with_categorical_features('Cadbury Perk - Chocolate Bar').values

pd.DataFrame({'Recommender1': recommender1,'Recommender2': recommender2})

,Recommender1,Recommender2
0,"X-press Instant Noodles - Masala Delight, Supe...",Nutties Chocolate Pack
1,1-2-3 Noodles - Pure Vegetarian,5 Star Chocolate Bar
2,Noodles - Chinese Hakka Veg,Dairy Milk Silk - Hazelnut Chocolate Bar
3,1-2-3 Noodles - Pure Vegetarian,"Perk - Chocolate, Home Treats, 175.5 g, 27 Units"
4,1-2-3 Noodles - Veg Masala Flavour,Dark Milk Chocolate Bar
5,1-2-3 Noodles - Chicken Flavour,Dairy Milk Silk Mousse - Chocolate Bar
6,1-2-3 Noodles - Veg Masala Flavour,Dark Milk Chocolate Bar
7,1-2-3 Noodles - Chicken Flavour,Chocolate Bar - Fuse
8,Noodles - Fiery Chilli,Choclairs Gold Coffee
9,FunFoods Pasta & Pizza Sauce,"5 Star Chocolate Home Pack, 200 g, 20 units"
